In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import shutil
import random

# Define paths
dataset1_path = "/kaggle/input/crop-and-weed-detection-data-with-bounding-boxes/agri_data/data/"
dataset2_base = "/kaggle/input/crop-weed-detection-for-yolov8/"
output_dir = "/kaggle/working/combined_dataset"  # Explicit absolute path for Kaggle
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")

# Create output directory structure
os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)
for dir_type in ["images", "labels"]:
    os.makedirs(os.path.join(train_dir, dir_type), exist_ok=True)
    os.makedirs(os.path.join(val_dir, dir_type), exist_ok=True)

# Function to copy files and avoid duplicates
def copy_files(src_images, src_labels, dst_images, dst_labels, prefix):
    image_extensions = ('.jpg', '.jpeg', '.png')
    image_files = [f for f in os.listdir(src_images) if f.lower().endswith(image_extensions)]
    label_files = [f for f in os.listdir(src_labels) if f.endswith('.txt')]
    
    print(f"Processing {os.path.basename(src_images)}: {len(image_files)} images, {len(label_files)} labels")
    
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]  # Keep full name up to last extension
        lbl_file = base_name + ".txt"
        if lbl_file in label_files:
            new_img_name = f"{prefix}_{img_file}"
            new_lbl_name = f"{prefix}_{lbl_file}"
            print(f"Copying: {img_file} -> {new_img_name}")
            shutil.copy(os.path.join(src_images, img_file), os.path.join(dst_images, new_img_name))
            shutil.copy(os.path.join(src_labels, lbl_file), os.path.join(dst_labels, new_lbl_name))
        else:
            print(f"Warning: No matching label for {img_file}")

# Dataset 1: Copy all files
print("Copying Dataset 1...")
copy_files(dataset1_path, dataset1_path, 
           os.path.join(output_dir, "images"), 
           os.path.join(output_dir, "labels"), "ds1")

# Dataset 2: Copy train, valid, and test
print("Copying Dataset 2...")
for split in ["train", "valid", "test"]:
    src_images = os.path.join(dataset2_base, split, "images")
    src_labels = os.path.join(dataset2_base, split, "labels")
    if os.path.exists(src_images) and os.path.exists(src_labels):
        copy_files(src_images, src_labels, 
                   os.path.join(output_dir, "images"), 
                   os.path.join(output_dir, "labels"), "ds2")
    else:
        print(f"Warning: {split} images or labels folder not found")

# Get all image files for splitting
images_dir = os.path.join(output_dir, "images")
labels_dir = os.path.join(output_dir, "labels")
image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Shuffle and split (e.g., 80% train, 20% val)
random.shuffle(image_files)
split_idx = int(0.8 * len(image_files))

train_images = image_files[:split_idx]
val_images = image_files[split_idx:]

# Function to move files
def move_files(file_list, src_images, src_labels, dst_images, dst_labels):
    for img_file in file_list:
        base_name = os.path.splitext(img_file)[0]
        lbl_file = base_name + ".txt"
        if os.path.exists(os.path.join(src_labels, lbl_file)):
            shutil.move(os.path.join(src_images, img_file), os.path.join(dst_images, img_file))
            shutil.move(os.path.join(src_labels, lbl_file), os.path.join(dst_labels, lbl_file))
        else:
            print(f"Warning: No matching label for {img_file}")

# Move train files
print("Moving train files...")
move_files(train_images, images_dir, labels_dir, 
           os.path.join(train_dir, "images"), os.path.join(train_dir, "labels"))

# Move val files
print("Moving val files...")
move_files(val_images, images_dir, labels_dir, 
           os.path.join(val_dir, "images"), os.path.join(val_dir, "labels"))

# Update data.yaml with correct paths
data_yaml = """train: /kaggle/working/combined_dataset/train/images
val: /kaggle/working/combined_dataset/val/images
nc: 2
names: ['crop', 'weed']
"""

with open(os.path.join(output_dir, "data.yaml"), "w") as f:
    f.write(data_yaml)

print("Datasets combined and split into train and val successfully. Check the combined_dataset folder!")

Copying Dataset 1...
Processing : 1300 images, 1300 labels
Copying: agri_0_7209.jpeg -> ds1_agri_0_7209.jpeg
Copying: agri_0_6978.jpeg -> ds1_agri_0_6978.jpeg
Copying: agri_0_9299.jpeg -> ds1_agri_0_9299.jpeg
Copying: agri_0_8085.jpeg -> ds1_agri_0_8085.jpeg
Copying: agri_0_8482.jpeg -> ds1_agri_0_8482.jpeg
Copying: agri_0_9584.jpeg -> ds1_agri_0_9584.jpeg
Copying: agri_0_9927.jpeg -> ds1_agri_0_9927.jpeg
Copying: agri_0_3398.jpeg -> ds1_agri_0_3398.jpeg
Copying: agri_0_7362.jpeg -> ds1_agri_0_7362.jpeg
Copying: agri_0_8682.jpeg -> ds1_agri_0_8682.jpeg
Copying: agri_0_82.jpeg -> ds1_agri_0_82.jpeg
Copying: agri_0_6222.jpeg -> ds1_agri_0_6222.jpeg
Copying: agri_0_2469.jpeg -> ds1_agri_0_2469.jpeg
Copying: agri_0_4383.jpeg -> ds1_agri_0_4383.jpeg
Copying: agri_0_4718.jpeg -> ds1_agri_0_4718.jpeg
Copying: agri_0_8209.jpeg -> ds1_agri_0_8209.jpeg
Copying: agri_0_84.jpeg -> ds1_agri_0_84.jpeg
Copying: agri_0_1317.jpeg -> ds1_agri_0_1317.jpeg
Copying: agri_0_3442.jpeg -> ds1_agri_0_3442.jpeg

In [2]:
!pip install -U ultralytics
!wandb disabled

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.6 MB/s eta 0:00:00:00:0100:01
  Attempting unins

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8s.pt')
model.train(data='/kaggle/working/combined_dataset/data.yaml', epochs=50, imgsz=640, batch=16, augment=True)

In [ ]:
from ultralytics import YOLO

model = YOLO('/kaggle/working/combined_dataset/runs/train/exp/weights/best.pt')  # Path to best model
results = model.predict(source='/kaggle/working/combined_dataset/val/images', save=True, conf=0.5)

In [ ]:
import os
import shutil
from IPython.display import FileLink

# Define the source directory
source_dir = "/kaggle/working/runs/detect/train"
output_zip = "/kaggle/working/train_metrics.zip"

# Create a zip file of the directory
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', source_dir)

# Generate a download link
FileLink(output_zip)

In [ ]:
model.export(format='onnx')  # Saves as best.onnx

In [ ]:
from ultralytics import YOLO

# Load the pre-trained model (medium version for better capacity)
model = YOLO('yolov8m.pt')

# Train with transfer learning
model.train(
    data='/kaggle/working/combined_dataset/data.yaml',
    epochs=100,              # Increase epochs to allow more learning
    imgsz=640,               # Keep 640 for detail
    batch=16,                # Maintain batch size, adjust if memory issues
    augment=True,            # Keep augmentation
    lr0=0.001,               # Lower learning rate for finer tuning
    patience=10,             # Early stopping if no improvement
    val=True,                # Validate during training
    project='/kaggle/working/runs/detect',  # Save runs here
    name='train_transfer'    # Unique name for this run
)

# Optional: Save the best model after training
best_model_path = '/kaggle/working/runs/detect/train_transfer/weights/best.pt'
model.save(best_model_path)
print(f"Best model saved at: {best_model_path}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 49.7M/49.7M [00:00<00:00, 180MB/s]


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/combined_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_transfer, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True

100%|██████████| 755k/755k [00:00<00:00, 17.9MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     30/100      6.43G       1.71      1.243      1.216          6        640: 100%|██████████| 207/207 [01:57<00:00,  1.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:09<00:00,  2.72it/s]

                   all        825       3521      0.594        0.6      0.601      0.308



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     31/100      6.46G      1.702      1.227      1.197        118        640:  35%|███▍      | 72/207 [00:41<01:17,  1.74it/s]

In [ ]:
from ultralytics import YOLO

model = YOLO('/kaggle/working/combined_dataset/runs/train/exp/weights/best.pt')  # Path to best model
results = model.predict(source='/kaggle/working/combined_dataset/val/images', save=True, conf=0.5)

In [ ]:
import os
import shutil
from IPython.display import FileLink

# Define the source directory
source_dir = "/kaggle/working/runs/detect/train"
output_zip = "/kaggle/working/train_metrics.zip"

# Create a zip file of the directory
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', source_dir)

# Generate a download link
FileLink(output_zip)

In [ ]:
model.export(format='onnx')  # Saves as best.onnx